# Résolution des sudokus 4x4

## Modélisation

- Une grille (potentiellement incomplète) va être représentée par une liste de 16 nombres.
- On stocke les cases par ligne puis colonne.
- Si une case est remplie on stocke son nombre
- Sinon on stocke l'opposé de son indice.

**Remarque** on utilise l'opposé des indices pour que si une répétition a lieu sur une ligne/colonne/cellule on est sûr que le problème viennent de nombre déjà rempli et pas de case vide.

## Exercice

- implémenter une fonction `pas_de_repetition`
- implémenter une fonction `verification_lignes`
- implémenter une fonction `verification_colonnes`
- implémenter une fonction `verification_cellules`
- implémenter une fonction `grille_suivante` (prenant une liste de grilles progressivement remplies).
- implémenter une fonction `grille_complete`

In [1]:
def repetition(liste):
    """Teste si au moins une valeur est répétée dans la liste"""
    for ind1, e1 in enumerate(liste):
        for ind2, e2 in enumerate(liste):
            if ind1 != ind2 and e1 == e2:
                return True
    return False

In [2]:
print("Faux : ", repetition([1, 2, 3, 4]))
print("Vrai : ", repetition([1, 2, 3, 4, 1]))

Faux :  False
Vrai :  True


In [3]:
print("Faux : ", repetition("abcde"))
print("Vrai : ", repetition("abcdedcba"))

Faux :  False
Vrai :  True


In [4]:
def verification_lignes(grille):
    """Vérifie qu'il n'y a pas de doublon sur aucune des lignes
    de la grille de sudoku 4x4
    """
    for debut in range(0, 16, 4):
        if repetition(grille[debut: debut+4]):
            return False
    return True
        

In [5]:
entree = list(range(0, 16))
print(entree)
print("Vrai : ", verification_lignes(entree))
entree[3] = 0
print(entree)
print("Faux : ", verification_lignes(entree))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Vrai :  True
[0, 1, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Faux :  False


In [6]:
def verification_colonnes(grille):
    """Vérifie qu'il n'y a pas de doublon sur aucune des colonnes
    de la grille de sudoku 4x4
    """
    for debut in range(0, 4):
        fin = debut + 16
        if repetition(grille[debut:fin:4]):
            return False
    return True

In [7]:
entree = list(range(0, 16))
print(entree)
print("Vrai : ", verification_colonnes(entree))
entree[8] = 0
print(entree)
print("Faux : ", verification_colonnes(entree))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Vrai :  True
[0, 1, 2, 3, 4, 5, 6, 7, 0, 9, 10, 11, 12, 13, 14, 15]
Faux :  False


**ATTENTION** on essaye de choisir les noms de variables et de fonctions pour décrire leur contenu/action de façon à ce qu'on ait l'impression de lire du pseudocode à leur utilisation.

In [8]:
def  verification_cellules(grille):
    """Verifie qu'il n'y ait pas de repetition dans les sous grilles
    2x2 de la grille de sudoku 4x4
    """
    base = 2
    if len(grille) != base ** 4:
        raise ValueError("La grille ne fait pas la bonne taille")
    for jj in range(0, base ** 4, base ** 3):
        for ii in range(0, base ** 2, base ** 1):
            if repetition(
                [
                    grille[ii + jj + i + j] 
                    for j in range(0, base ** 3, base ** 2) 
                    for i in range(0, base ** 1, base ** 0)
                ]
            ):
                return False
    return True

In [9]:
entree = list(range(0, 16))
print(entree)
print("Vrai : ", verification_cellules(entree))
entree[15] = 10
print(entree)
print("Faux : ", verification_cellules(entree))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Vrai :  True
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 10]
Faux :  False


## implémentation alternative

plus lisible moins facilement généralisable

In [10]:
def verification(grille, liste_indices):
    """Vérification de manière brutale"""
    for indices in liste_indices:
        if repetition([grille[indice] for indice in indices]):
            return False
    return True

In [11]:
regle_cellules =  [
    (0, 1, 4, 5), 
    (2, 3, 6, 7), 
    (8, 9, 12, 13),
    (10, 11, 14, 15),
]
regle_lignes = [
    (0, 1, 2, 3),
    (4, 5, 6, 7),
    (8, 9, 10, 11),
    (12, 13, 14, 15),
]
regle_colonnes = [
    (0, 4, 8, 12),
    (1, 5, 9, 13),
    (2, 6, 10, 14),
    (3, 7, 11, 15),
]

## Implémenter le solveur de sudoku

In [12]:
def est_valide(grille):
    """Fonction de validation d'une grille intermédiaire"""
    if not verification_lignes(grille):
        return False
    if not verification_colonnes(grille):
        return False
    if not verification_cellules(grille):
        return False
    return True

In [13]:
def case_suivante(grille):
    """Retourne l'indice de la prochaine case à remplir
    """
    for indice_case, contenu_case in enumerate(grille):
        if contenu_case <= 0:
            return indice_case
    return None

In [83]:
def solveur(grille_depart):
    """Solveur de sudoku 4x4. Premier jet.
    """
    grille_courante = grille_depart[:]
    cases_modifiees = list()
    drapeau = True
    while True:
        if drapeau and est_valide(grille_courante):
            indice_prochaine_case = case_suivante(grille_courante)
            if indice_prochaine_case is not None:
                grille_courante[indice_prochaine_case] = 1
                cases_modifiees.append(indice_prochaine_case)
            else:
                return grille_courante
        else:
            drapeau = True
            if cases_modifiees:
                indice_derniere_case = cases_modifiees.pop()
                if grille[indice_derniere_case] == 4:
                    grille[indice_derniere_cases] = -indice_derniere_case
                    drapeau = False
                else:
                    grille[indice_derniere_case] = grille[indice_derniere_case] + 1
            else:
                raise ValueError("Grille de départ sans solution")
                #return None

## Exercice

- Tester sur quelques cas la fonction précédente (terminaison? validité?)
- Coder le sudoku 9x9
- Modifier le code pour renvoyer toutes les solutions


In [84]:
def affichage(grille):
    """Genere une version plus lisible de la grille de sudoku sous forme de chaine de caractères"""
    filtree = list()
    for nbr in grille:
        if nbr > 0:
            filtree.append(nbr)
        else:
            filtree.append(" ")
    return """
_________________
| {} | {} | {} | {} |
-----------------
| {} | {} | {} | {} |
-----------------
| {} | {} | {} | {} |
-----------------
| {} | {} | {} | {} |
-----------------
    """.format(*filtree)

In [85]:
complete = [1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
print(affichage(complete))
print(est_valide(complete))


_________________
| 1 | 2 | 3 | 4 |
-----------------
| 3 | 4 | 1 | 2 |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    
True


In [86]:
essai1 = complete[:]
for indice in (0,):
    essai1[indice] = -indice
print(affichage(essai1))
print(affichage(solveur(essai1)))


_________________
|   | 2 | 3 | 4 |
-----------------
| 3 | 4 | 1 | 2 |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    

_________________
| 1 | 2 | 3 | 4 |
-----------------
| 3 | 4 | 1 | 2 |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    


In [87]:
essai1 = complete[:]
for indice in (0, 3):
    essai1[indice] = -indice
print(affichage(essai1))
print(affichage(solveur(essai1)))


_________________
|   | 2 | 3 |   |
-----------------
| 3 | 4 | 1 | 2 |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    


NameError: name 'grille' is not defined

On a trouvé une faute de frappe, modifiée dans la version suivante

In [92]:
def solveur(grille_depart):
    """Solveur de sudoku 4x4
    Version faute de frappe corrigée, print de débugage ajoutés.
    """
    grille_courante = grille_depart[:]
    cases_modifiees = list()
    drapeau = True
    while True:
        print("grille courante : ", grille_courante)
        print("case modifiees : ", cases_modifiees)
        print("drapeau : ", drapeau)
        print()
        if drapeau and est_valide(grille_courante):
            indice_prochaine_case = case_suivante(grille_courante)
            if indice_prochaine_case is not None:
                grille_courante[indice_prochaine_case] = 1
                cases_modifiees.append(indice_prochaine_case)
            else:
                return grille_courante
        else:
            drapeau = True
            if cases_modifiees:
                indice_derniere_case = cases_modifiees.pop()
                if grille_courante[indice_derniere_case] == 4:
                    grille_courante[indice_derniere_cases] = -indice_derniere_case
                    drapeau = False
                else:
                    grille_courante[indice_derniere_case] = grille_courante[indice_derniere_case] + 1
            else:
                raise ValueError("Grille de départ sans solution")
                #return None

In [93]:
essai1 = complete[:]
for indice in (0, 3):
    essai1[indice] = -indice
print(affichage(essai1))
print(affichage(solveur(essai1)))


_________________
|   | 2 | 3 |   |
-----------------
| 3 | 4 | 1 | 2 |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    
grille courante :  [0, 2, 3, -3, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  []
drapeau :  True

grille courante :  [1, 2, 3, -3, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0]
drapeau :  True

grille courante :  [1, 2, 3, 1, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 2, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0]
drapeau :  True

grille courante :  [2, 2, 3, 2, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  []
drapeau :  True



ValueError: Grille de départ sans solution

Problème sur l'implémentation la grille ayant une solution. Résolu grâce au print.

In [94]:
def solveur(grille_depart):
    """Solveur de sudoku 4x4
    Version complète.
    """
    grille_courante = grille_depart[:]
    cases_modifiees = list()
    drapeau = True
    while True:
        print("grille courante : ", grille_courante)
        print("case modifiees : ", cases_modifiees)
        print("drapeau : ", drapeau)
        print()
        if drapeau and est_valide(grille_courante):
            indice_prochaine_case = case_suivante(grille_courante)
            if indice_prochaine_case is not None:
                grille_courante[indice_prochaine_case] = 1
                cases_modifiees.append(indice_prochaine_case)
            else:
                return grille_courante
        else:
            drapeau = True
            if cases_modifiees:
                indice_derniere_case = cases_modifiees.pop()
                if grille_courante[indice_derniere_case] == 4:
                    grille_courante[indice_derniere_cases] = -indice_derniere_case
                    drapeau = False
                else:
                    grille_courante[indice_derniere_case] = grille_courante[indice_derniere_case] + 1
                    cases_modifiees.append(indice_derniere_case)
            else:
                raise ValueError("Grille de départ sans solution")
                #return None

In [95]:
essai1 = complete[:]
for indice in (0, 3, 7):
    essai1[indice] = -indice
print(affichage(essai1))
print(affichage(solveur(essai1)))


_________________
|   | 2 | 3 |   |
-----------------
| 3 | 4 | 1 |   |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
| 4 | 3 | 2 | 1 |
-----------------
    
grille courante :  [0, 2, 3, -3, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  []
drapeau :  True

grille courante :  [1, 2, 3, -3, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0]
drapeau :  True

grille courante :  [1, 2, 3, 1, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 2, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 3, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 4, 3, 4, 1, -7, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 4, 3, 4, 1, 1, 2, 1, 4, 3, 4, 3, 2, 1]
case modifiees :  [0, 3, 7]
drapeau :  True

grille courante :  [1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4,

In [96]:
essai1 = complete[:]
for indice in (0, 3, 7, 12, 15):
    essai1[indice] = -indice
print(affichage(essai1))
print(affichage(solveur(essai1)))


_________________
|   | 2 | 3 |   |
-----------------
| 3 | 4 | 1 |   |
-----------------
| 2 | 1 | 4 | 3 |
-----------------
|   | 3 | 2 |   |
-----------------
    
grille courante :  [0, 2, 3, -3, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  []
drapeau :  True

grille courante :  [1, 2, 3, -3, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0]
drapeau :  True

grille courante :  [1, 2, 3, 1, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 2, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 3, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 4, 3, 4, 1, -7, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0, 3]
drapeau :  True

grille courante :  [1, 2, 3, 4, 3, 4, 1, 1, 2, 1, 4, 3, -12, 3, 2, -15]
case modifiees :  [0, 3, 7]
drapeau :  True

grille courante :  [1, 2, 3, 4